In [0]:
%python
## Import libraries
import pyspark as ps
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *
import pandas as pd
import json

In [0]:
# Retrieve the parameter values
job_id = dbutils.widgets.get("job_id")
usecase_id = dbutils.widgets.get("usecase_id")

# Print the parameter values
print(f"job_id: {job_id}")
print(f"usecase_id: {usecase_id}")

In [0]:
class Error_Handle:
  def __init__(self):
    self.query = "Query About to fill"
    self.sql_df = pd.DataFrame()
    self.data_json = {"Data":[]}    
    self.usecase_id = None
  
  def logger(self, message):
    print(message)
    
  def set_query_string(self, usecase_parm):
    query = f"""with m(usecase_id,hist_retention) as (SELECT USECASE_ID, hist_retention FROM com_us_alyt_ngebox.metadata_usecase WHERE USECASE_ID = '{usecase_parm}'),
s(NGEBox_Record_ID,SUGGEST_EXTERNAL_ID_VOD__C) as (select p.NGEBox_Record_ID, p.SUGGEST_EXTERNAL_ID_VOD__C from (select * from com_us_alyt_ngebox.ngebox_suggestion_requests_processed where USECASE_ID = '{usecase_parm}') p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where p.usecase_id in (select usecase_id from m) and (s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0)) ),
p(usecase_id,NGEBox_Record_ID ) as (select pt.usecase_id,pt.NGEBox_Record_ID from com_us_alyt_ngebox.ngebox_suggestion_requests_processed pt where pt.USECASE_ID = '{usecase_parm}' and pt.ownerid is null and pt.usecase_id in (511,512) and CAST(pt.created_date as DATE) >= date_add(current_date, -(select m.hist_retention from m)) and pt.SUGGEST_EXTERNAL_ID_VOD__C in (select distinct SUGGEST_EXTERNAL_ID_VOD__C from s where pt.NGEBox_Record_ID = s.NGEBox_Record_ID ))
merge into com_us_alyt_ngebox.ngebox_suggestion_requests_processed process_table using ( select usecase_id,NGEBox_Record_ID from p ) source_table on ( process_table.USECASE_ID = '{usecase_parm}' and process_table.usecase_id = source_table.usecase_id and process_table.NGEBox_Record_ID = source_table.NGEBox_Record_ID ) when matched then update set process_table.Not_Sent_To_Veeva_Reason = CASE WHEN process_table.Not_Sent_To_Veeva_Reason IS NULL THEN 'ownerid is null' ELSE CONCAT(process_table.Not_Sent_To_Veeva_Reason, '|', 'ownerid is null') END;

with m(usecase_id,hist_retention) as (SELECT USECASE_ID, hist_retention FROM com_us_alyt_ngebox.metadata_usecase WHERE USECASE_ID = '{usecase_parm}'),
s(NGEBox_Record_ID,SUGGEST_EXTERNAL_ID_VOD__C) as (select p.NGEBox_Record_ID, p.SUGGEST_EXTERNAL_ID_VOD__C from (select * from com_us_alyt_ngebox.ngebox_suggestion_requests_processed where USECASE_ID = '{usecase_parm}') p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where p.USECASE_ID = '{usecase_parm}' and  p.usecase_id in (select usecase_id from m) and (s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0)) ),
p(usecase_id,NGEBox_Record_ID ) as (select pt.usecase_id,pt.NGEBox_Record_ID from com_us_alyt_ngebox.ngebox_suggestion_requests_processed pt where pt.USECASE_ID = '{usecase_parm}' and pt.HCP_OMNI_ID is null  and pt.usecase_id in (511) and CAST(pt.created_date as DATE) >= date_add(current_date, -(select m.hist_retention from m)) and pt.SUGGEST_EXTERNAL_ID_VOD__C in (select distinct SUGGEST_EXTERNAL_ID_VOD__C from s where pt.NGEBox_Record_ID = s.NGEBox_Record_ID ))
merge into com_us_alyt_ngebox.ngebox_suggestion_requests_processed process_table using ( select usecase_id,NGEBox_Record_ID from p ) source_table on (process_table.USECASE_ID = '{usecase_parm}' and process_table.usecase_id = source_table.usecase_id and process_table.NGEBox_Record_ID = source_table.NGEBox_Record_ID ) when matched then update set process_table.Not_Sent_To_Veeva_Reason = CASE WHEN process_table.Not_Sent_To_Veeva_Reason IS NULL THEN 'Omni_id is null' ELSE CONCAT(process_table.Not_Sent_To_Veeva_Reason, '|', 'Omni_id is null') END;

with m(usecase_id,hist_retention) as (SELECT USECASE_ID, hist_retention FROM com_us_alyt_ngebox.metadata_usecase WHERE USECASE_ID = '{usecase_parm}'),
s(NGEBox_Record_ID,SUGGEST_EXTERNAL_ID_VOD__C) as (select p.NGEBox_Record_ID, p.SUGGEST_EXTERNAL_ID_VOD__C from (select * from com_us_alyt_ngebox.ngebox_suggestion_requests_processed where USECASE_ID = '{usecase_parm}') p left join com_us_lake.rep_suggestion_vod__c s on p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C where p.USECASE_ID = '{usecase_parm}' and  p.usecase_id in (select usecase_id from m) and (s.SUGGESTION_EXTERNAL_ID_VOD__C is NULL or (s.SUGGESTION_EXTERNAL_ID_VOD__C is not null and s.DISMISSED_VOD__C = 0)) ),
p(usecase_id,NGEBox_Record_ID ) as (select pt.usecase_id,pt.NGEBox_Record_ID from com_us_alyt_ngebox.ngebox_suggestion_requests_processed pt where pt.USECASE_ID = '{usecase_parm}' and pt.hcp_npi is null and pt.usecase_id in (511) and CAST(pt.created_date as DATE) >= date_add(current_date, -(select m.hist_retention from m)) and pt.SUGGEST_EXTERNAL_ID_VOD__C in (select distinct SUGGEST_EXTERNAL_ID_VOD__C from s where pt.NGEBox_Record_ID = s.NGEBox_Record_ID ))
merge into com_us_alyt_ngebox.ngebox_suggestion_requests_processed process_table using ( select usecase_id,NGEBox_Record_ID from p ) source_table on (process_table.USECASE_ID = '{usecase_parm}' and process_table.USECASE_ID <> 511 and process_table.usecase_id = source_table.usecase_id and process_table.NGEBox_Record_ID = source_table.NGEBox_Record_ID ) when matched then update set process_table.Not_Sent_To_Veeva_Reason = CASE WHEN process_table.Not_Sent_To_Veeva_Reason IS NULL THEN 'NPI is null' ELSE CONCAT(process_table.Not_Sent_To_Veeva_Reason, '|', 'NPI is null') END;"""
    self.query = query    

  def run_query_and_set_sqldf(self, usecase_id):
      self.usecase_id = usecase_id
      query_list = self.query.split(";")
      self.sql_df = []  
      for curr_query in query_list:
          if curr_query.strip() == "":
              continue
          curr_query_with_param = curr_query.replace("Usecase_parm", usecase_id)
          sql_df = sqlContext.sql(curr_query_with_param).collect()
          self.sql_df.extend(sql_df)
    
  def format_sqldf_to_json(self):
    sql_output_rowlist = []
    for curr_row in self.sql_df: 
      sql_output_rowlist.append (curr_row.asDict(True))
    sql_df_new = pd.DataFrame(sql_output_rowlist)
    sql_jsonstr = sql_df_new.to_json(orient="records") # type(result) -> str
    sql_jsonparse = json.loads(sql_jsonstr) # type(parsed_json) -> list
    nb_json = {"data":sql_jsonparse} # type(metadata_json) -> dict
    self.nb_json = nb_json
    
  def construct_microservice_response(self):
    resp = {
      "nge_response": {
        "status": 200,
        "body": self.nb_json
      }
    }
    self.response = resp

In [0]:
usecase_id = dbutils.widgets.get("usecase_id")
nb_obj = Error_Handle()
nb_obj.set_query_string(usecase_parm=usecase_id)
nb_obj.run_query_and_set_sqldf(usecase_id=usecase_id)
nb_obj.format_sqldf_to_json()
nb_obj.construct_microservice_response()
dbutils.notebook.exit(nb_obj.response)